# Project Anomaly Finder API Example

### This Jupyter notebook shows you how to get started with the Project Anomaly Finder API in Python, and how to visualize your results.

In [13]:
import requests
import json
import pandas as pd
import numpy as np
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')

# Import library to display results
import matplotlib.pyplot as plt
%matplotlib inline 

In [14]:
from bokeh.plotting import figure,output_notebook, show
from bokeh.palettes import Blues4
from bokeh.models import ColumnDataSource,Slider
import datetime
from bokeh.io import push_notebook
from dateutil import parser
from ipywidgets import interact, widgets, fixed
output_notebook()

Loading BokehJS ...

In [15]:
def detect(endpoint, subscription_key, request_data):
    headers = {'Content-Type': 'application/json', 'Ocp-Apim-Subscription-Key': subscription_key}
    response = requests.post(endpoint, data=json.dumps(request_data), headers=headers)
    if response.status_code == 200:
        return json.loads(response.content.decode("utf-8"))
    else:
        print(response.status_code)
        raise Exception(response.text)

In [16]:
def build_figure(sample_data, senstivity):
    sample_data['Sensitivity'] = senstivity
    result = detect(endpoint, subscription_key, sample_data)
    columns = {'ExpectedValues': result['ExpectedValues'], 'IsAnomaly': result['IsAnomaly'], 'IsNegativeAnomaly': result['IsNegativeAnomaly'],
          'IsPositiveAnomaly': result['IsPositiveAnomaly'], 'UpperMargins': result['UpperMargins'], 'LowerMargins': result['LowerMargins'],
          'Timestamp': [parser.parse(x['Timestamp']) for x in sample_data['Series']], 
          'Value': [x['Value'] for x in sample_data['Series']]}
    response = pd.DataFrame(data=columns)
    values = response['Value']
    label = response['Timestamp']
    anomalies = []
    anomaly_labels = []
    index = 0
    anomaly_indexes = []
    p = figure(x_axis_type='datetime', title="Anomaly Finder Result ({0} Sensitvity)".format(senstivity), width=800, height=600)
    for anom in response['IsAnomaly']:
        if anom == True and (values[index] > response.iloc[index]['ExpectedValues'] + response.iloc[index]['UpperMargins'] or 
                         values[index] < response.iloc[index]['ExpectedValues'] - response.iloc[index]['LowerMargins']):
            anomalies.append(values[index])
            anomaly_labels.append(label[index])
            anomaly_indexes.append(index)
        index = index+1
    upperband = response['ExpectedValues'] + response['UpperMargins']
    lowerband = response['ExpectedValues'] -response['LowerMargins']
    band_x = np.append(label, label[::-1])
    band_y = np.append(lowerband, upperband[::-1])
    boundary = p.patch(band_x, band_y, color=Blues4[2], fill_alpha=0.5, line_width=1, legend='Boundary')
    p.line(label, values, legend='Value', color="#2222aa", line_width=1)
    p.line(label, response['ExpectedValues'], legend='ExpectedValue',  line_width=1, line_dash="dotdash", line_color='olivedrab')
    anom_source = ColumnDataSource(dict(x=anomaly_labels, y=anomalies))
    anoms = p.circle('x', 'y', size=5, color='tomato', source=anom_source)
    p.legend.border_line_width = 1
    p.legend.background_fill_alpha  = 0.1
    show(p, notebook_handle=True)

### Find anomalies of sample timeseries in batch

Anomaly Finder API returns default result on whether a data point is anomaly or not, and the upper and lower bound can be calculated from ExpectedValue and UpperMargin/LowerMargin. Those default values should work fine for most cases. However, some scenarios require different bounds than the default ones. The recommend practice is applying a MarginScale on the UpperMargin or LowerMargin to adjust the dynamic bounds. UpperBoundary equals to ExpectedValue + (100 - MarginScale)\*UpperMargin, lowerBoundary equals to ExpectedValue - (100-MarginScale)\*LowerMargin

In [17]:
endpoint = 'https://westus2.api.cognitive.microsoft.com/anomalyfinder/v2.0/timeseries/entire/detect'
subscription_key = '' #Here you have to paste your primary key

In [18]:
# Hourly Sample
sample_data = json.load(open('sample_hourly.json'))
sample_data['Granularity'] = 'hourly'
sample_data['Period'] = 24
# 95 Sensitivity
build_figure(sample_data,95)

In [19]:
# 90 Sensitivity
build_figure(sample_data,90)

In [20]:
#85 Sensitivity
build_figure(sample_data,85)

In [21]:
#daily sample
sample_data = json.load(open('sample.json'))
sample_data['Granularity'] = 'daily'
# 95 Sensitivity
build_figure(sample_data,95)

In [22]:
# 90 Sensitivity
build_figure(sample_data,90)

In [23]:
# 85 Sensitivity
build_figure(sample_data,80)